# Installation H2 

Le système utilisé pendant les TME est H2.

In [13]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Télécharger le pilote de H2

In [14]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-04-13 00:09:22--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1,0M) [application/octet-stream]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    in 0,01s   

2024-04-13 00:09:22 (98,7 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]

--2024-04-13 00:09:22--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2543011 (2,4M) [application/octet-stream]
Saving to: ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  --.-KB/s    in 0,03s   

2024-04-13 00:09:

**Relancez le kernel**: Kernel -> Restart kernel ...

https://www.psycopg.org/docs/
http://localhost:8082

In [15]:
import psycopg2
import os
local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

## Démarrer le serveur SQL H2

In [16]:
#Remplacez les valeurs des variables suivantes
#NN= numéro de groupe: chaine de caractères (ex: 01, 02,.., 13), remplacer xx 
NN = '06'
#BB=numero binome: chaîne de caractères (assigné par l'enseignant, ex: 01,..., 15), remplacer xx
BB = '13'
port = 5000+int(NN+BB)
print(f'Le numero du port utilisé pour la connexion à la BD est: {port}')

Le numero du port utilisé pour la connexion à la BD est: 5613


In [17]:
%%bash --bg --out output -s "$port"
java -Dh2.bindAddress=127.0.0.1 -cp h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort $1 -baseDir ./data -ifNotExists

# Fonctions utiles

## connect_H2

In [18]:
def connect_H2(db,user,password):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

## execute

In [19]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount:
            names = [desc[0] for desc in cursor.description]
            for attr in names:
                print(str(attr[:16]).ljust(16),end='|')
            print()
            for attr in names:
                print(''.ljust(17,'-'),end='')
            print()
            for ligne in cursor:
                for attr in ligne:
                    print(str(attr)[:16].ljust(16),end='|')
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

**TD/TME 6 : REQUÊTES IMBRIQUÉES AVEC EXISTS, ALL ET ANY**

Ce TD/TME utilise les données contenues dans le fichier **bd_jo_v2_H2.sql**

**Attention**: vous pouvez cliquer sur les 3 lignes dans la fenêtre de gauche pour d'afficher les différentes sections du notebook 

# TD6

On considère le schéma de la base JEUXOLYMPIQUE2014 qui décrit les athlètes et leurs résultats aux
épreuves des sports des Jeux Olympiques d'Hiver Sotchi 2014 :

**PAYS** ( <u>CODEPAYS</u>, NOMP)<br/>
Ex. ('FRA', 'France')<br/>
**SPORT** ( <u>SID</u>, NOMSP)<br/>
Ex. (1, 'Biathlon')<br/>
**EPREUVE** ( <u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN)<br/>
Ex. (10, 1, 'relais 4x7,5km', 'Hommes', 22/02/2014, 22/02/2014)<br/>
**ATHLETE** ( <u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)<br/>
Ex. (1000, 'SOBOLEV', 'Alexey', NULL, 'RUS')<br/>
**EQUIPE** ( <u>EQID</u>, CODEPAYS*)<br/>
Ex. (30, 'SUI')<br/>
**ATHLETESEQUIPE** ( <u>EQID*, AID*</u>)<br/>
Ex. (30, 796) : L'athlète (aid=796) a participé à l'équipe (eqid=30)<br/>
**RANGINDIVIDUEL** ( <u>EPID*, AID*</u>, RANG)<br/>
Ex. (15, 61, 1) : L'athlète (aid=61) a gagné la médaille d'or (rang=1) de l'épreuve (epid=15)<br/>
**RANGEQUIPE** ( <u>EPID*, EQID*</u>, RANG)<br/>
Ex. (10, 30, 14) : L'équipe (eqid=30) a été classée 14e à l'épreuve (epid=10)<br/>


Les attributs qui forment la clé primaire de chaque relation sont soulignés. Les clés étrangères sont
signalées avec une *. Les attributs aid, epid, eqid et sid correspondent aux identifiants des athlètes,
épreuves, équipes et sports et sont utilisés à la fois comme clé primaire ou comme référence (clé
étrangère) vers la relation correspondante.
La relation **PAYS** contient le code et le nom de tous les pays, même si ils n'ont pas participé aux
Jeux Olympiques. Les sports (n-uplets de la relation **SPORT**) sont un ensemble d'épreuves (n-uplets
de la relation **EPREUVE**). Pour chaque épreuve on connaît son nom et les date de début et fin de
l'épreuve. Les épreuves peuvent être individuelles ou par équipe. Dans le premier cas, la
participation des athlètes (n-uplets de la relation ATHLETE) est stocké dans la table
**RANGINDIVIDUEL** qui contient en plus le rang qu'ils ont obtenu (1 pour la médaille d'or). Pour les
épreuves par équipe les résultats sont stockés dans la relation **RANGEQUIPE**, alors que l'information
sur le pays de chaque équipe et ses participants et stocké dans les relations **EQUIPE** et
**ATHLETESEQUIPE**. Dans les relations **RANGINDIVIDUEL** et **RANGEQUIPE** l'attribut rang est égal à
null si l'athlète ou l'équipe a été disqualifié.




## Connexion à H2

In [20]:
connection = connect_H2("jo"+f'{port}',"ba","ba")

Connection closed
Connection to H2 DB successful


## Créer les tables et charger les données bd_jo_v2_H2.sql

In [84]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


**Requêtes: ANY /ALL / IN / EXISTS**

## **Q1**. 
Les athlètes ayant participé à (au moins) une épreuve individuelle et (au moins) une épreuve par équipe. Exprimer la requête en utilisant ANY (au lieu de IN; voir requête 2.6 dans TD5).
Résultat : (372 lignes)

In [25]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel r1
where a.aid = r1.aid and a.aid = any (select aEq.aid from athletesEquipe aEq, rangEquipe r2 where aEq.eqid = r2.eqid)

"""
execute(connection,query,show=False)

372 rows


<cursor object at 0x7f82e9b6c8b0; closed: -1>

 ## **Q2. a)** 
Les pays ayant au moins un athlète (requête équivalente: les pays ayant participé aux JO).
Résultat : (88 lignes)

In [27]:
query="""
select p.nomp
from pays p
where p.codepays = any (select a.codepays from athlete a)

"""
execute(connection,query,show=False)

88 rows


<cursor object at 0x7f82e9b6d120; closed: -1>

##  **Q2. b**) 
Les pays ayant exactement un seul athlète
Résultat : (18 lignes)

In [29]:
query="""
select p.nomp
from pays p, athlete a
where p.codepays = a.codepays and not exists (select * from athlete a2 where a2.codepays = p.codepays and a2.aid <> a.aid)

"""
execute(connection,query,show=True)

18 rows
nomp            |
-----------------
Bermudes        |
Îles Caïmans    |
Hong Kong       |
Îles Vierges des|
Îles Vierges bri|
Kirghizistan    |
Luxembourg      |
Mexique         |
Malte           |
Népal           |
Pakistan        |
Paraguay        |
Philippines     |
Tonga           |
Tadjikistan     |
Timor oriental  |
Venezuela       |
Zimbabwe        |


<cursor object at 0x7f82e9b6d210; closed: -1>

## **Q3. a)** 
Les athlètes qui n'ont jamais été disqualifiés aux épreuves individuelles.
Résultat : (2194 lignes)

In [33]:
query="""
select distinct a.nomath, a.prenomath
from athlete a
where a.aid not in (select a2.aid from athlete a2, rangIndividuel r where r.aid = a2.aid and r.rang is null)

"""
execute(connection,query,show=False)

2194 rows


<cursor object at 0x7f82e9b6d8a0; closed: -1>

## **Q3. b)** 
Les pays qui n'ont pas eu d'athlète disqualifié aux épreuves individuelles.
Résultat : (143 lignes)

In [36]:
query="""
select p.nomp
from pays p
where p.codepays not in (select a.codepays from athlete a, rangIndividuel r where r.aid = a.aid and r.rang is null)
"""
execute(connection,query,show=False)

143 rows


<cursor object at 0x7f82e9b6e020; closed: -1>

##  **Q3. c)** 
Les pays qui ont participé aux JO et qui n'ont pas eu d'athlètes disqualifiés aux épreuves
individuelles.
Résultat : (25 lignes)

In [42]:
query="""
select distinct p.nomp
from pays p, athlete a2
where p.codepays = a2.codepays and p.codepays not in (select a.codepays from athlete a, rangIndividuel r where r.aid = a.aid and r.rang is null)
"""
execute(connection,query,show=True)

25 rows
nomp            |
-----------------
Belgique        |
Bermudes        |
Danemark        |
Estonie         |
Hong Kong       |
Inde            |
Iran            |
Jamaïque        |
Kazakhstan      |
Kirghizistan    |
Liban           |
Luxembourg      |
Malte           |
Mongolie        |
Monténégro      |
Népal           |
Pakistan        |
Paraguay        |
Pays-Bas        |
Philippines     |
Taipei chinois  |
Timor oriental  |
Tonga           |
Îles Vierges bri|
Îles Vierges des|


<cursor object at 0x7f82e9b6e3e0; closed: -1>

##  **Q3. d)** 
Les pays qui ont participé aux JO et qui n'ont pas eu des athlètes disqualifiés ni en
individuel ni par équipe
Résultat : (25 lignes)

In [48]:
query="""
select distinct p.nomp
from pays p, athlete a2
where p.codepays = a2.codepays and p.codepays not in (select a.codepays from athlete a, rangIndividuel r where r.aid = a.aid and r.rang is null)
and p.codepays not in (select eq.codepays from rangEquipe r, Equipe eq where r.eqid = eq.eqid and r.rang is null)
"""
execute(connection,query,show=True)

25 rows
nomp            |
-----------------
Belgique        |
Bermudes        |
Danemark        |
Estonie         |
Hong Kong       |
Inde            |
Iran            |
Jamaïque        |
Kazakhstan      |
Kirghizistan    |
Liban           |
Luxembourg      |
Malte           |
Mongolie        |
Monténégro      |
Népal           |
Pakistan        |
Paraguay        |
Pays-Bas        |
Philippines     |
Taipei chinois  |
Timor oriental  |
Tonga           |
Îles Vierges bri|
Îles Vierges des|


<cursor object at 0x7f82e9b6ea70; closed: -1>

##  **Q4. a)** 
Les athlètes n'ayant pas gagné de médaille ni en individuel ni en équipe
Résultat : (1921 lignes)

In [52]:
query="""
select a.nomath, a.prenomath
from athlete a
where a.aid not in (select a2.aid from athlete a2, rangIndividuel ri where a2.aid = ri.aid and ri.rang<4)
and a.aid not in (select aEq.aid from athletesEquipe aEq, rangEquipe rEq where aEq.eqid = rEq.eqid and rEq.rang<4)

"""
execute(connection,query,show=False)

1921 rows


<cursor object at 0x7f82e9b6f4c0; closed: -1>

##  **Q4. b)** 
Les pays ayant participé aux JO et n'ayant pas gagné de médaille ni en individuel ni en
équipe.
Résultat : (63 lignes)

In [55]:
query="""
select distinct p.nomp
from pays p, athlete a
where p.codepays = a.codepays and p.codepays not in (select distinct a2.codepays from athlete a2, rangIndividuel ri where a2.aid = ri.aid and ri.rang<4)
and p.codepays not in (select eq.codepays from equipe eq, rangEquipe rEq where eq.eqid = rEq.eqid and rEq.rang<4)
"""
execute(connection,query,show=True)


63 rows
nomp            |
-----------------
Albanie         |
Andorre         |
Argentine       |
Arménie         |
Azerbaïdjan     |
Belgique        |
Bermudes        |
Bosnie-Herzégovi|
Brésil          |
Bulgarie        |
Chili           |
Chypre          |
Danemark        |
Dominique       |
Espagne         |
Estonie         |
Ex-république yo|
Grèce           |
Géorgie         |
Hong Kong       |
Hongrie         |
Inde            |
Iran            |
Irlande         |
Islande         |
Israël          |
Jamaïque        |
Kirghizistan    |
Liban           |
Liechtenstein   |
Lituanie        |
Luxembourg      |
Malte           |
Maroc           |
Mexique         |
Moldavie        |
Monaco          |
Mongolie        |
Monténégro      |
Nouvelle-Zélande|
Népal           |
Ouzbékistan     |
Pakistan        |
Paraguay        |
Philippines     |
Portugal        |
Pérou           |
Roumanie        |
Saint-Marin     |
Serbie          |
Slovaquie       |
Tadjikistan     |
Taipei chinois  |
Th

<cursor object at 0x7f82e9b6fa60; closed: -1>

##  **Q5.** 
L'(es) épreuve(s) avec la plus grande durée.
Résultat : Hockey sur glace, Hockey sur glace, Hommes, 16 jours

In [68]:
query="""
select s.nomsp, ep.nomep, ep.categorie, DATEDIFF(day,ep.datedebut,ep.datefin)+1 as duree
from epreuve ep, sport s
where DATEDIFF(day, ep.dateDebut, ep.dateFin) >= ALL (SELECT DATEDIFF(day, e2.dateDebut, e2.dateFin) FROM Epreuve e2)
"""
execute(connection,query,show=True)

15 rows
nomsp           |nomep           |categorie       |duree           |
--------------------------------------------------------------------
Biathlon        |hockey sur glace|Hommes          |16              |
Bobsleigh       |hockey sur glace|Hommes          |16              |
Combiné nordique|hockey sur glace|Hommes          |16              |
Curling         |hockey sur glace|Hommes          |16              |
Hockey sur glace|hockey sur glace|Hommes          |16              |
Luge            |hockey sur glace|Hommes          |16              |
Patinage artisti|hockey sur glace|Hommes          |16              |
Patinage de vite|hockey sur glace|Hommes          |16              |
Patinage de vite|hockey sur glace|Hommes          |16              |
Saut à ski      |hockey sur glace|Hommes          |16              |
Skeleton        |hockey sur glace|Hommes          |16              |
Ski acrobatique |hockey sur glace|Hommes          |16              |
Ski alpin       |hockey su

<cursor object at 0x7f82e9b987c0; closed: -1>

##  **Q6.**  
Les athlètes ayant gagné une médaille à toutes les épreuve individuelles auxquelles ils ont participé. Résultat : (109 lignes)

In [73]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel r2
where a.aid=r2.aid and 4 > all (select r.rang from rangIndividuel r where r.aid = a.aid)

"""
execute(connection,query,show=False)

109 rows


<cursor object at 0x7f82e9b98b80; closed: -1>

**<u>BASE FOOFLE</u>**

## Créer les tables et charger les données base_foofle_H2.sql

In [74]:
schemafile=open("base_foofle_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


##  **Q7.**  
Quelles équipes ont déjà joué au stade préféré de l'équipe des Piepla ? Résultats : (2 lignes) Direkt , Piepla

In [75]:
query="""
select distinct m.eq1
from match m
where m.st = (select stp from equipe where neq = 'Piepla')

"""
execute(connection,query,show=True)

2 rows
eq1             |
-----------------
Direkt          |
Piepla          |


<cursor object at 0x7f82e9b98f40; closed: -1>

##  **Q8.**  
Quels sont les stades où a déjà joué Manon Messi ? Résultats: (3 lignes) GrandArena, Boulodrome, BukHall

In [76]:
query="""
select distinct m.st
from match m
where m.eq1 = (select eq from joueur where njo = 'Manon Messi')

"""
execute(connection,query,show=True)

3 rows
st              |
-----------------
Boulodrome      |
BukHall         |
GrandArena      |


<cursor object at 0x7f82e9b99120; closed: -1>

##  **Q9.**  
A quelle date a eu lieu un match entre deux équipes sponsorisées par le même sponsor ? Résultats :(4 lignes) 2015-05-16, 2015-05-15, 2015-06-15, 2015-05-12

In [79]:
query="""
select distinct m.datem
from match m, joueur j, sponsorise s
where j.eq = m.eq1 and s.njo = j.njo and s.nsp = any (select s2.nsp from sponsorise s2, joueur j2 where j2.eq = m.eq2 and s2.njo = j.njo)

"""
execute(connection,query,show=True)

6 rows
datem           |
-----------------
2015-05-12      |
2015-05-13      |
2015-05-15      |
2015-05-16      |
2015-06-12      |
2015-06-15      |


<cursor object at 0x7f82e9b99030; closed: -1>

##  **Q10.**  
Quel sponsor a financé deux joueurs différents ayant eu un match le même jour et dans des
stades différents mais proches (moins de 50 km) ? Résultats :(4 lignes) Air Monaco, Carouf, Robek, Adadis

In [82]:
query="""
select distinct s.nsp
from sponsorise s,match m1, joueur j1
where m1.eq1 = j1.eq and j1.njo = s.njo and s.nsp = any (select s2.nsp from sponsorise s2,match m2, joueur j2, distance d where m1.datem = m2.datem and d.st1 = m1.st and d.st2 = m2.st and d.nbkm<50 and m2.eq1 = j2.eq and j2.njo = s2.njo)
 
"""
execute(connection,query,show=True)

4 rows
nsp             |
-----------------
Adadis          |
Air Monaco      |
Carouf          |
Robek           |


<cursor object at 0x7f82e9b994e0; closed: -1>

# TME6

On considère le schéma de la base JEUXOLYMPIQUE2014 donné en TD :

**PAYS** ( <u>CODEPAYS</u>, NOMP)<br/>
Ex. ('FRA', 'France')<br/>
**SPORT** ( <u>SID</u>, NOMSP)<br/>
Ex. (1, 'Biathlon')<br/>
**EPREUVE** ( <u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN)<br/>
Ex. (10, 1, 'relais 4x7,5km', 'Hommes', 22/02/2014, 22/02/2014)<br/>
**ATHLETE** ( <u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)<br/>
Ex. (1000, 'SOBOLEV', 'Alexey', NULL, 'RUS')<br/>
**EQUIPE** ( <u>EQID</u>, CODEPAYS*)<br/>
Ex. (30, 'SUI')<br/>
**ATHLETESEQUIPE** ( <u>EQID*, AID*</u>)<br/>
Ex. (30, 796) : L'athlète (aid=796) a participé à l'équipe (eqid=30)<br/>
**RANGINDIVIDUEL** ( <u>EPID*, AID*</u>, RANG)<br/>
Ex. (15, 61, 1) : L'athlète (aid=61) a gagné la médaille d'or (rang=1) de l'épreuve (epid=15)<br/>
**RANGEQUIPE** ( <u>EPID*, EQID*</u>, RANG)<br/>
Ex. (10, 30, 14) : L'équipe (eqid=30) a été classée 14e à l'épreuve (epid=10)<br/>

**Requêtes: ANY /ALL / IN / EXISTS**

## **Q1.** 
Les sports dont toutes les épreuves ont duré un seul jour.
Résultat : Ski de fond, Ski alpin, Biathlon

In [85]:
query="""
select s.nomsp
from sport s
where s.sid not in (select distinct ep.sid from epreuve ep where datediff(day,ep.datedebut,ep.datefin)>0)

"""
execute(connection,query,show=True)

3 rows
nomsp           |
-----------------
Biathlon        |
Ski alpin       |
Ski de fond     |


<cursor object at 0x7f82e9b99b70; closed: -1>

## **Q2.** 
Les sports qui n'ont pas d'épreuves de categorie Mixte.
Résultat : (12 lignes)

In [87]:
query="""
select s.nomsp
from sport s
where s.sid not in (select distinct ep.sid from epreuve ep where ep.categorie = 'Mixte')
"""
execute(connection,query,show=True)

12 rows
nomsp           |
-----------------
Bobsleigh       |
Combiné nordique|
Curling         |
Hockey sur glace|
Patinage de vite|
Patinage de vite|
Saut à ski      |
Skeleton        |
Ski acrobatique |
Ski alpin       |
Ski de fond     |
Surf des neiges |


<cursor object at 0x7f82e9b996c0; closed: -1>

## **Q3.** 
Les équipes dont aucun athlète n'a gagné de médaille aux épreuves individuelles.
Attention : il y a des équipes sans athlètes.
Résultat : (265 lignes avec les équipes sans athlètes - 252 lignes sans les équipes sans
athlètes)

Avec les équipes sans athlètes :

In [98]:
query="""
select eq.eqid
from equipe eq
where eq.eqid not in (select aEq.eqid from athletesEquipe aEq, rangindividuel ri where ri.rang<4 and ri.aid = aEq.aid)
"""
execute(connection,query,show=False)

265 rows


<cursor object at 0x7f82e9ba07c0; closed: -1>

Sans les équipes sans athlètes :

In [99]:
query="""
select distinct eq.eqid
from equipe eq, athletesEquipe aEq2
where aEq2.eqid = eq.eqid and eq.eqid not in (select aEq.eqid from athletesEquipe aEq, rangindividuel ri where ri.rang<4 and ri.aid = aEq.aid)

"""
execute(connection,query,show=False)

252 rows


<cursor object at 0x7f82e9ba08b0; closed: -1>

## **Q4.** 
La nationalité de l'athlète le/la plus jeune. Attention : il y a des athlètes dont on ne connaît pas la date de naissance.
Résultat : ('29/11/1998','JPN')

In [106]:
query="""
select a.datenaissance, p.nomp
from athlete a, pays p
where a.codepays = p.codepays and a.datenaissance >= all (select a2.datenaissance from athlete a2 where a2.datenaissance is not null)

"""
execute(connection,query,show=True)

1 rows
datenaissance   |nomp            |
----------------------------------
1998-11-29      |Japon           |


<cursor object at 0x7f82e9ba0c70; closed: -1>

## **Q5.** 
Le plus jeune athlète de chaque pays.
Résultat : (26 lignes)

In [108]:
query="""
select a.datenaissance, p.nomp
from athlete a, pays p
where a.codepays = p.codepays and a.datenaissance is not null and a.datenaissance >= all (select a2.datenaissance from athlete a2 where a2.datenaissance is not null and a2.codepays = a.codepays)

"""
execute(connection,query,show=True)

26 rows
datenaissance   |nomp            |
----------------------------------
1986-12-27      |Australie       |
1992-02-25      |Autriche        |
1986-08-03      |Biélorussie     |
1995-12-05      |Canada          |
1996-06-03      |République popul|
1979-11-23      |Croatie         |
1993-04-28      |République tchèq|
1992-01-12      |Finlande        |
1995-11-03      |France          |
1988-10-31      |Royaume-Uni     |
1995-08-28      |Allemagne       |
1992-10-10      |Italie          |
1998-11-29      |Japon           |
1993-06-13      |Kazakhstan      |
1997-01-30      |Corée du Sud    |
1990-08-21      |Lettonie        |
1991-12-09      |Pays-Bas        |
1994-07-02      |Norvège         |
1989-10-05      |Pologne         |
1998-06-05      |Russie          |
1992-09-20      |Slovénie        |
1991-04-27      |Suisse          |
1984-08-28      |Slovaquie       |
1993-06-24      |Suède           |
1990-09-19      |Ukraine         |
1995-08-17      |États-Unis      |


<cursor object at 0x7f82e9ba0d60; closed: -1>

# Fermer la connexion

In [109]:
connection.commit() # implicit avec close
connection.close()

In [110]:
connection

<connection object at 0x7f82e9fc5440; dsn: 'user=ba password=xxx dbname=jo5613 host=localhost port=5613', closed: 1>